In [ ]:
!pip install --upgrade --quiet kuzu
## the extensions will install to ~/.kuzu/extension/0.10.0/win_amd64/xxx  refer https://extension.kuzudb.com/v0.10.0/win_amd64/xxx
import kuzu


# Create an empty on-disk database and connect to it
db = kuzu.Database("./demo_db_0.10.0")
print(db.get_version())
print(db.get_storage_version())
conn = kuzu.Connection(db)
# Create a graph with a node label and a property
try:
    conn.execute(
        """
          CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name));
          CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name));
          CREATE REL TABLE ActedIn (FROM Person TO Movie);
          CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'});
          CREATE (:Person {name: 'Robert De Nero', birthDate: '1943-08-17'});
          CREATE (:Movie {name: 'The Godfather: Part II'});
          MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m);
          MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Nero' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m);
        """
    )
except Exception as e:
    print(f"Error executing graph operations: {str(e)}")
# Execute Cypher query
response = conn.execute(
    """
INSTALL neo4j;
LOAD neo4j;
    """
)
while response.has_next():
    print(response.get_next())
response.close()
conn.close()
db.close()

0.10.0
38
Error executing graph operations: Binder exception: Movie already exists in catalog.


RuntimeError: IO exception: Failed to download extension: neo4j at URL extension.kuzudb.com/v0.10.0/win_amd64/neo4j/libneo4j.kuzu_extension (ERROR: Failed to read connection)

In [ ]:
import kuzu


class KuzuSession:
    def __init__(self, db_path):
        self.db_path = db_path

    def __enter__(self):
        self.db = kuzu.Database(self.db_path)
        self.conn = kuzu.Connection(self.db)
        return self.conn

    def __exit__(self, exc_type, exc_val, exc_tb):
       # self.conn.close()
       # self.db.close()
        print("Session closed successfully.")

# Usage:
with KuzuSession("demo_db") as conn:
    result = conn.execute("MATCH (n) RETURN n LIMIT 1")
    print(result.get_as_df())

In [ ]:
from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_db") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 10")
    print(df)

In [ ]:

from safe_kuzu_subprocess import SafeKuzuSubprocess
with SafeKuzuSubprocess("demo_large") as db:
    df = db.execute("MATCH (n)-[r]-(m) RETURN * LIMIT 2000")
    print(df)